# Folders generation for processing DONKEY
This notebook uses the simulated and real data to produce the folders for training and evaluation

## Usage:
Run all cells

## Requirements:
./content/datasets/data_in/objects/ must contain folders /sim/ & /real/ each with the sim and real images and driving logs.

## Outputs:
./content/datasets/processed_data/real \
./content/datasets/processed_data/sim


In [ ]:
import h5py
import numpy as np
import PIL.Image as Image
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint
import h5py
import numpy as np
import PIL.Image as Image
import os
import cv2
import matplotlib.pyplot as plt

import re
from collections import OrderedDict
import json

img = np.array(Image.open("./content/datasets/data_in/objects/real/26_cam-image_array_.jpg"))
obj_ids = np.unique(img)
print(obj_ids)

def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]

def load_data(images_folder):
  raw_image=[]
  image_files = sorted(os.listdir(images_folder),key=natural_sort_key)
  for image_file in image_files:
      image = cv2.imread(images_folder+image_file)
      raw_image.append(image)
  return raw_image


raw_image_sim = []
raw_image_real = []


print("-STARTING real- ")
images_folder = f"./content/datasets/data_in/objects/real/"
raw_image_real=load_data(images_folder)

print("-STARTING sim- ")
images_folder = f"./content/datasets/data_in/objects/sim/"
raw_image_sim=load_data(images_folder)

In [ ]:
for i in range(0, 100,1):
      print(i)
      # Create a 2x2 grid of subplots
      fig, axs = plt.subplots(1, 2, figsize=(15, 6))
      plt.subplots_adjust(wspace=0.2, hspace=0.4)

      # Plot the real and fake images side by side in the first row
      axs[0].imshow(cv2.cvtColor(raw_image_real[i], cv2.COLOR_BGR2RGB))
      axs[0].set_title('Real Image')
      axs[1].imshow(cv2.cvtColor(raw_image_sim[i], cv2.COLOR_BGR2RGB))
      axs[1].set_title('Sim Image')
      for ax in axs.flat:
          ax.set_xticks([])
          ax.set_yticks([])

      # Show the plot for the current iteration
      plt.show()

In [ ]:
dataset_index="image"
def find_lowest_dimensions(images_list1, images_list2):
    # Find the lowest height and width across both lists
    lowest_height = min(images_list1[0].shape[0], images_list2[0].shape[0])
    lowest_width = min(images_list1[0].shape[1], images_list2[0].shape[1])
    return lowest_height, lowest_width

def crop_images_to_lowest_dimensions(images_list, lowest_height, lowest_width):
    cropped_images = [image[:lowest_height, :lowest_width,:] for image in images_list]
    return cropped_images

def crop_1d_to_lowest_dimensions(images_list, lowest_height, lowest_width):
    cropped_images = [image[:lowest_height, :lowest_width] for image in images_list]
    return cropped_images

lowest_height=10000
lowest_width=10000
height, width = find_lowest_dimensions(raw_image_real, raw_image_sim)
lowest_height=min(height,lowest_height)
lowest_width=min(width,lowest_width)
print(lowest_height,lowest_width)



In [ ]:

loaded_dictionary_images_real={}
loaded_dictionary_images_sim={}

loaded_dictionary_images_real[dataset_index] = crop_images_to_lowest_dimensions(raw_image_real, lowest_height, lowest_width)
loaded_dictionary_images_sim[dataset_index] = crop_images_to_lowest_dimensions(raw_image_sim, lowest_height, lowest_width)

train_indexes_gan = {}
test_indexes_gan = {}


# pattern = 'ttvttvttvt'
pattern = 'tttttttttt'
pattern_pointer = 0
train_dataset_indices_inner = []
test_dataset_indices_inner = []

for index in range(0, min(len(loaded_dictionary_images_real[dataset_index]),len(loaded_dictionary_images_sim[dataset_index]))):
    pattern_char = pattern[pattern_pointer]
    if pattern_char == 't':
        train_dataset_indices_inner.append(index)
    else:
        test_dataset_indices_inner.append(index)
    pattern_pointer = (pattern_pointer + 1) % len(pattern)
train_indexes_gan[dataset_index]=train_dataset_indices_inner
test_indexes_gan[dataset_index]=test_dataset_indices_inner

print("GAN train and test")
print("Dataset", dataset_index)
print("Train: ",len(train_indexes_gan[dataset_index]))
print("Test: ",len(test_indexes_gan[dataset_index]))



In [ ]:
print(dataset_index)
for i in range(0, 100):
  # Create a 2x2 grid of subplots
  fig, axs = plt.subplots(1, 2, figsize=(15, 6))
  plt.subplots_adjust(wspace=0.2, hspace=0.4)

  # Plot the real and fake images side by side in the first row
  axs[0].imshow(cv2.cvtColor(loaded_dictionary_images_real[dataset_index][i], cv2.COLOR_BGR2RGB))
  axs[0].set_title('Real Image')
  axs[1].imshow(cv2.cvtColor(loaded_dictionary_images_sim[dataset_index][i], cv2.COLOR_BGR2RGB))
  axs[1].set_title('Sim Image')

  for ax in axs.flat:
      ax.set_xticks([])
      ax.set_yticks([])

  # Show the plot for the current iteration
  plt.show()

In [8]:
def save_sim_real_outputs(real_path,sim_path):
          dataset_index="image"
          print(dataset_index)
          counter=0
          for i in train_indexes_gan[dataset_index]:
                counter+=1
                input_image_real = np.array(loaded_dictionary_images_real[dataset_index][i])
                input_image_real = tf.convert_to_tensor(input_image_real.astype(np.uint8), np.uint8)
                input_image_real = tf.reverse(input_image_real, axis=[-1])
                
                png_image = tf.image.encode_png(input_image_real)
                os.makedirs('./content/datasets/processed_data/'+real_path+'/', exist_ok=True)
                with open('./content/datasets/processed_data/'+real_path+'/'+str(dataset_index)+'_'+str(i)+'.png', 'wb') as f:
                  f.write(png_image.numpy())
                
        
                input_image_sim = np.array(loaded_dictionary_images_sim[dataset_index][i])
                input_image_sim = tf.convert_to_tensor(input_image_sim.astype(np.uint8), np.uint8)
                input_image_sim = tf.reverse(input_image_sim, axis=[-1])
                png_image = tf.image.encode_png(input_image_sim)
                os.makedirs('./content/datasets/processed_data/'+sim_path+'/', exist_ok=True)
                with open('./content/datasets/processed_data/'+sim_path+'/'+str(dataset_index)+'_'+str(i)+'.png', 'wb') as f:
                  f.write(png_image.numpy())
                

                # error_lane=0
                  

                # error_data_sim = {
                #     "sim_real": error_lane,
                # }

                # output_folder = "./content/output_plots/"+sim_path+"_lane_error/"
                # os.makedirs(output_folder, exist_ok=True)
                # output_path = output_folder + str(dataset_index) + "_" + str(i) +".json"
                # with open(output_path, "w") as json_file:
                #     json.dump(error_data_sim, json_file, indent=4)

                
                

In [ ]:
save_sim_real_outputs("real","sim")